In [ ]:
import requests
import pandas as pd
def get_median_score_and_distance(directory):
    ids = []
    total_df = pd.DataFrame()
    with open(directory, 'r') as f:
        df = pd.read_csv(f, sep='\t', header=None)
    for _, row in df.iterrows():
        ids.append(row[0].split(',')[0])
    for id in ids:
        data = requests.get(url=f'https://geoguessr.com/api/v3/games/{id}').json()
        score = []
        distance = []
        for guess in data['player']['guesses']:
            score.append(guess['roundScore']['amount'])
            distance_ = float(guess['distance']['meters']['amount'])
            if guess['distance']['meters']['unit'] == 'm':
                distance_ /= 1000
            distance.append(distance_)
        df = pd.DataFrame({'id': id, 'score': score, 'distance': distance})
        total_df = pd.concat([total_df, df])
    total_df['score'] = total_df['score'].astype(int)
    total_df['distance'] = total_df['distance'].astype(float)
    print('geoguessr score:')
    print(f"median: {round(total_df['score'].median(),2)}")
    print(f"mean: {round(total_df['score'].mean(),2)}")
    print('\ndistance in km:')
    print(f"median: {round(total_df['distance'].median(),2)}")
    print(f"mean: {round(total_df['distance'].mean(),2)}")
    return total_df.reset_index(drop=True)
print('four imgs\n')
df_four_imgs = get_median_score_and_distance('/Users/sebastian/Downloads/final_score_aow.txt')
print('------------------------------------')
print('mean embd\n')
df_mean_embd = get_median_score_and_distance('/Users/sebastian/Downloads/final_score_aow_mean.txt')

In [110]:
import requests
import pandas as pd
import pycountry

def get_session(ncfa_token):
    BASE_URL = "https://www.geoguessr.com/api/" 
    _ncfa_TOKEN = ncfa_token
    session = requests.Session()
    session.cookies.set("_ncfa", _ncfa_TOKEN, domain="www.geoguessr.com")

    profiles = session.get(f"{BASE_URL}v3/profiles")
    if profiles.status_code != 200:
        Exception(f"Error: {profiles.status_code}")
    return session

df_metadata = pd.read_csv('../data/imbag_metadata.csv')

country_dict = {country.alpha_2.lower(): country.name for country in pycountry.countries}
country_dict['ru'] = 'Russia'
country_dict['bo'] = 'Bolivia'
country_dict['cw'] = 'Curacao'
country_dict['cz'] = 'Czech Republic'
country_dict['la'] = 'Laos'
country_dict['re'] = 'Reunion'
country_dict['kr'] = 'South Korea'
country_dict['sz'] = 'Swaziland'
country_dict['tw'] = 'Taiwan'
country_dict['vi'] = 'U.S. Virgin Islands'

token = r'lQ3WYhyUoftGEuqJRUIP3uTF%2BlYewaXMc3bvkjBA1eo%3DBzWlf4bDQsA6mwj0I7bklqRlLhG%2F77KQxiKbjbLpLnhtaO1tVorpRricStcxJuJKCa4up5CycmSptBlb%2BMK0Cy7m4NWcJcp2U0GRn%2F2Jc%2F4%3D'
sess = get_session(token)
game_id = '17fd8361-c203-49d8-956a-25c4d5184649'


In [111]:
def compare_team_duels(sess, game_id, country_dict, df_metadata):
    data = sess.get(f'https://game-server.geoguessr.com/api/duels/{game_id}').json()

    country_truth = []

    country_guess_1 = []
    distance_guess_1 = []
    score_guess_1 = []

    country_guess_2 = []
    distance_guess_2 = []
    score_guess_2 = []


    for rounds in data['rounds']:
        image = rounds['panorama']
        #print(f"{rounds['roundNumber']}:\t{image['lat']}\t{image['lng']}\t{country_dict.get(image['countryCode'])}")
        country_truth.append(country_dict.get(image['countryCode']))


    for player in range(2):
        for guess in data['teams'][player]['roundResults']:
            roundNumber = guess['roundNumber']
    #        if isinstance(guess['bestGuess'], dict) == False:
    #            print(f'no guess from player {player+1} on round {roundNumber}')
            #else:
            score = guess['score']
            lat = guess['bestGuess']['lat']
            lng = guess['bestGuess']['lng']
            country, region = df_metadata[(df_metadata['Latitude']==lat) & (df_metadata['Longitude']==lng)].iloc[0][['Country', 'Region']]
            distance = guess['bestGuess']['distance']
            #print(f'player {player+1} on round {roundNumber}: score: {score}, lat: {lat}, lng: {lng}, distance: {distance}, country: {country}, region: {region}')
            if player == 0:
                country_guess_1.append(country)
                distance_guess_1.append(distance)
                score_guess_1.append(score)

            elif player == 1:
                country_guess_2.append(country)
                distance_guess_2.append(distance)
                score_guess_2.append(score)


                
    df = pd.DataFrame({'country_truth': country_truth[:len(country_guess_1)], 'country_guess_1': country_guess_1, 'distance_guess_1': distance_guess_1, 'score_guess_1': score_guess_1, 'country_guess_2': country_guess_2, 'distance_guess_2': distance_guess_2, 'score_guess_2': score_guess_2})
    df['distance_guess_1'] = df['distance_guess_1'].astype(float)
    df['score_guess_1'] = df['score_guess_1'].astype(int)
    df['distance_guess_2'] = df['distance_guess_2'].astype(float)
    df['score_guess_2'] = df['score_guess_2'].astype(int)
    print('player 1:')
    print(f"median distance: {round(df['distance_guess_1'].median(),2)}")
    print(f"mean distance: {round(df['distance_guess_1'].mean(),2)}")
    print(f"median score: {round(df['score_guess_1'].median(),2)}")
    print(f"mean score: {round(df['score_guess_1'].mean(),2)}")

    print('\nplayer 2:')
    print(f"median distance: {round(df['distance_guess_2'].median(),2)}")
    print(f"mean distance: {round(df['distance_guess_2'].mean(),2)}")
    print(f"median score: {round(df['score_guess_2'].median(),2)}")
    print(f"mean score: {round(df['score_guess_2'].mean(),2)}")
    return df

In [119]:
df = compare_team_duels(sess, game_id, country_dict, df_metadata)
df

player 1:
median distance: 209870.2
mean distance: 459438.22
median score: 4442.0
mean score: 4013.44

player 2:
median distance: 208006.52
mean distance: 628136.25
median score: 4447.0
mean score: 4006.19


,country_truth,country_guess_1,distance_guess_1,score_guess_1,country_guess_2,distance_guess_2,score_guess_2
0,United Arab Emirates,United Arab Emirates,3.544544e+03,4990,United Arab Emirates,6.862624e+04,4810
1,Russia,Russia,7.569314e+05,3263,Russia,6.175582e+05,3530
2,Cambodia,Philippines,2.352748e+06,1327,Philippines,2.408788e+06,1286
3,Turkey,Turkey,1.287751e+06,2419,Turkey,1.287751e+06,2419
4,Lithuania,Hungary,9.452577e+05,2935,Hungary,8.914759e+05,3025
5,Brazil,Brazil,1.697623e+05,4544,Brazil,1.697623e+05,4544
6,Canada,Canada,2.777749e+05,4275,Canada,2.899510e+05,4246
7,New Zealand,New Zealand,9.364423e+04,4743,New Zealand,9.364423e+04,4743
8,Turkey,Austria,1.550216e+06,2087,Austria,1.811615e+06,1801
9,Kyrgyzstan,Kyrgyzstan,2.039823e+05,4457,Kyrgyzstan,2.039823e+05,4457
